In [1]:
import sys
sys.path.insert(0, '../')
from variant.varianthdf5 import VariantHDF5

%load_ext autoreload
%autoreload 2

In [2]:
VARIANT_FILE_PATH = '/home/k/Jumis/guardiome/genome-data/customers/738/variant/738.concat_snp_indel.extract_chr.recode.ann_snpeff.ann_clinvar.rename_chr.vcf.gz'
variant_hdf5 = VariantHDF5(VARIANT_FILE_PATH)

Reading variant HDF5 ...
Reading ID-to-chromosome dict ...


In [4]:
variant_hdf5.get_variants_by_id('rs662799')

[{'ALT': 'A',
  'CHROM': '11',
  'GT': 'None',
  'ID': 'rs662799',
  'QUAL': 1181.77,
  'REF': 'G',
  'effect': 'upstream_gene_variant',
  'end': 116792991,
  'gene_name': 'ZPR1',
  'impact': 'MODIFIER',
  'pathogenicity': '',
  'start': 116792991,
  'variant_type': 'SNP'}]

In [16]:
variant_hdf5.get_variants_by_region('MT', 0, 30000)

Reading chromosome_MT_variants where: (0 <= start) & (end <= 30000) ...


[{'ALT': 'G',
  'CHROM': 'MT',
  'GT': 'None',
  'ID': '.',
  'QUAL': 99820.797,
  'REF': 'A',
  'effect': '',
  'end': 73,
  'gene_name': '',
  'impact': 'MODIFIER',
  'pathogenicity': '',
  'start': 73,
  'variant_type': 'SNP'},
 {'ALT': 'C',
  'CHROM': 'MT',
  'GT': 'None',
  'ID': 'rs117135796',
  'QUAL': 165728.0,
  'REF': 'T',
  'effect': '',
  'end': 152,
  'gene_name': '',
  'impact': 'MODIFIER',
  'pathogenicity': '',
  'start': 152,
  'variant_type': 'SNP'},
 {'ALT': 'G',
  'CHROM': 'MT',
  'GT': 'None',
  'ID': '.',
  'QUAL': 60607.801,
  'REF': 'A',
  'effect': '',
  'end': 263,
  'gene_name': '',
  'impact': 'MODIFIER',
  'pathogenicity': '',
  'start': 263,
  'variant_type': 'SNP'},
 {'ALT': 'C',
  'CHROM': 'MT',
  'GT': 'None',
  'ID': '.',
  'QUAL': 53830.699,
  'REF': 'CA',
  'effect': '',
  'end': 286,
  'gene_name': '',
  'impact': 'MODIFIER',
  'pathogenicity': '',
  'start': 286,
  'variant_type': 'DEL'},
 {'ALT': 'AC,ACC',
  'CHROM': 'MT',
  'GT': 'None',
  'ID': 